# Telco Customer Churn - Preprocessing

## 1. Load Libraries

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import math
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
import os



## 2. Load and Inspect Data

In [2]:
df = pd.read_csv('../data/raw/Telco-Customer-Churn.csv')


## 3. Data Cleaning

In [3]:
# Convert TotalCharges to numeric, coercing errors to NaN
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

# Check for missing values
print(df.isnull().sum())

# Fill missing TotalCharges with 0 (assuming new customers) 
# Checking rows with null TotalCharges
display(df[df['TotalCharges'].isnull()])

# It seems these are customers with tenure=0. We can fill with 0.
df['TotalCharges'] = df['TotalCharges'].fillna(0)

# Drop customerID as it is not needed for prediction
df.drop('customerID', axis=1, inplace=True)

customerID           0
gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
MultipleLines        0
InternetService      0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
PaperlessBilling     0
PaymentMethod        0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
488,4472-LVYGI,Female,0,Yes,Yes,0,No,No phone service,DSL,Yes,...,Yes,Yes,Yes,No,Two year,Yes,Bank transfer (automatic),52.55,NaN,No
753,3115-CZMZD,Male,0,No,Yes,0,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.25,NaN,No
936,5709-LVOEQ,Female,0,Yes,Yes,0,Yes,No,DSL,Yes,...,Yes,No,Yes,Yes,Two year,No,Mailed check,80.85,NaN,No
1082,4367-NUYAO,Male,0,Yes,Yes,0,Yes,Yes,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.75,NaN,No
1340,1371-DWPAZ,Female,0,Yes,Yes,0,No,No phone service,DSL,Yes,...,Yes,Yes,Yes,No,Two year,No,Credit card (automatic),56.05,NaN,No
3331,7644-OMVMY,Male,0,Yes,Yes,0,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,19.85,NaN,No
3826,3213-VVOLG,Male,0,Yes,Yes,0,Yes,Yes,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.35,NaN,No
4380,2520-SGTTA,Female,0,Yes,Yes,0,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.00,NaN,No
5218,2923-ARZLG,Male,0,Yes,Yes,0,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,One year,Yes,Mailed check,19.70,NaN,No
6670,4075-WKNIU,Female,0,Yes,Yes,0,Yes,Yes,DSL,No,...,Yes,Yes,Yes,No,Two year,No,Mailed check,73.35,NaN,No


In [4]:
print(df.isnull().sum())

gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64


## 4 Outlier Identification

#### 4.1 Sigma Rule

In [5]:
def find_anomaly(data):
    mean = data.mean()
    std = data.std()

    upper_bound = mean + (3 * std)
    lower_bound = mean - (3 * std)

    return (data > upper_bound) | (data < lower_bound)

num_cols = ['tenure', 'MonthlyCharges', 'TotalCharges']

for col in num_cols:
    n_outliers = find_anomaly(df[col]).sum()
    n_outlier_percentage = round(n_outliers / len(df),3)
    print(f"{col} : {n_outliers} outliers ({n_outlier_percentage} %)")

tenure : 0 outliers (0.0 %)
MonthlyCharges : 0 outliers (0.0 %)
TotalCharges : 0 outliers (0.0 %)


#### 4.2 IQR Method

In [6]:
def find_anomaly(data):
    Q1 = np.percentile(data, 25)
    Q3 = np.percentile(data, 75)

    IQR = Q3 - Q1 

    upper_bound = Q3 + 1.5 * IQR
    lower_bound = Q1 - 1.5 * IQR 

    return (data > upper_bound) | (data < lower_bound)
for col in num_cols:
    n_outliers = find_anomaly(df[col]).sum()
    n_outlier_percentage = round(n_outliers / len(df), 3)
    print(f"{col} : {n_outliers} outliers ({n_outlier_percentage} %)")

tenure : 0 outliers (0.0 %)
MonthlyCharges : 0 outliers (0.0 %)
TotalCharges : 0 outliers (0.0 %)


## 5 Feature Engineering

In [7]:
# Tenure Categories
def tenure_group(t):
    if t <= 12: return 'New'
    elif t <= 48: return 'Established'
    else: return 'Loyal'

df['TenureCategory'] = df['tenure'].apply(tenure_group)
print(df['TenureCategory'].value_counts())

TenureCategory
Established    2618
Loyal          2239
New            2186
Name: count, dtype: int64


In [8]:
# Determine columns to scale and encode
target = 'Churn'
numeric_features = ['tenure', 'MonthlyCharges', 'TotalCharges']
categorical_features = [c for c in df.columns if c not in numeric_features and c != target]

print(f"Numeric: {numeric_features}")
print(f"Categorical: {categorical_features}")

Numeric: ['tenure', 'MonthlyCharges', 'TotalCharges']
Categorical: ['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'TenureCategory']


In [9]:
# Encoding
# Binary encoding for Churn
le = LabelEncoder()
df[target] = le.fit_transform(df[target])

# One-Hot Encoding for other categoricals
df_processed = pd.get_dummies(df, columns=categorical_features, drop_first=False, dtype=int)

# Display processed dataframe
df_processed.head()

,tenure,MonthlyCharges,TotalCharges,Churn,gender_Female,gender_Male,SeniorCitizen_0,SeniorCitizen_1,Partner_No,Partner_Yes,...,Contract_Two year,PaperlessBilling_No,PaperlessBilling_Yes,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,TenureCategory_Established,TenureCategory_Loyal,TenureCategory_New
0,1,29.85,29.85,0,1,0,1,0,0,1,...,0,0,1,0,0,1,0,0,0,1
1,34,56.95,1889.50,0,0,1,1,0,1,0,...,0,1,0,0,0,0,1,1,0,0
2,2,53.85,108.15,1,0,1,1,0,1,0,...,0,0,1,0,0,0,1,0,0,1
3,45,42.30,1840.75,0,0,1,1,0,1,0,...,0,1,0,1,0,0,0,1,0,0
4,2,70.70,151.65,1,1,0,1,0,1,0,...,0,0,1,0,0,1,0,0,0,1


## 6. Split Dataset

In [10]:
# Split into X and y
X = df_processed.drop(target, axis=1)
y = df_processed[target]

# Train Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Feature Scaling
scaler = StandardScaler()
X_train[numeric_features] = scaler.fit_transform(X_train[numeric_features])
X_test[numeric_features] = scaler.transform(X_test[numeric_features])

print(f"Train shape: {X_train.shape}")
print(f"Test shape: {X_test.shape}")

Train shape: (5634, 49)
Test shape: (1409, 49)


In [11]:
# Save processed data
os.makedirs('../data/processed', exist_ok=True)

train_data = pd.concat([X_train, y_train], axis=1)
test_data = pd.concat([X_test, y_test], axis=1)

train_data.to_csv('../data/processed/train.csv', index=False)
test_data.to_csv('../data/processed/test.csv', index=False)

print("Data saved to data/processed")

Data saved to data/processed


In [12]:
train_data.head()

,tenure,MonthlyCharges,TotalCharges,gender_Female,gender_Male,SeniorCitizen_0,SeniorCitizen_1,Partner_No,Partner_Yes,Dependents_No,...,PaperlessBilling_No,PaperlessBilling_Yes,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,TenureCategory_Established,TenureCategory_Loyal,TenureCategory_New,Churn
3738,0.102371,-0.521976,-0.262257,0,1,1,0,1,0,1,...,1,0,0,0,1,0,1,0,0,0
3151,-0.711743,0.337478,-0.503635,0,1,1,0,0,1,0,...,1,0,0,0,0,1,1,0,0,0
4860,-0.793155,-0.809013,-0.749883,0,1,1,0,0,1,0,...,1,0,0,0,0,1,1,0,0,0
3867,-0.263980,0.284384,-0.172722,1,0,1,0,0,1,1,...,0,1,0,1,0,0,1,0,0,0
3810,-1.281624,-0.676279,-0.989374,0,1,1,0,0,1,0,...,1,0,0,0,1,0,0,0,1,0
